In [ ]:
##Importing packages
import numpy as np
from scipy.optimize import minimize
import cv2
import pandas as pd

import matplotlib.pyplot as plt
import random
import pandas as pd

import os
import copy
import json

from collections import Counter
import plotly.graph_objects as go

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
# functions needed
# rodrigues - for converting rotation vector to rotation matrix to be used while projection and reprojection
# project - for converting 3d points to 2d points using camera and object matrices 
# reprojection error - loss/objective function to be minimised
# bundle adjustment - kind of a wrapper function performing entire bundle adjustment functionality

In [ ]:
##function to convert rotation vector to rotation matrix
## r = 3X1 vector
def rodrigues(r):
    R, _ = cv2.Rodrigues(r)
    return R

In [ ]:
##function to project 3d points global world coordinate to 2d image pixel coordinate
##this will project all points *for a single frame* assuming for now that a single frame has single object (which is also moving) only (in case of multiple objects (which is also moving), different extrinsic matrices will be used accoridngly)
##points (np.ndarray) == (num_keypoints X 3); params (tuple) = (K_camera(3X3), R_camera(3X1), 
# t_camera(3X1), R_object(3X1), t_object(3X1) )
##outputs (np.ndarray) == (num_keypoints X 2) 10 keypoints x,y coordinates

 

def project(points, params):
    points=points.T  #3X10
    ##convert points to homogenous format (linearising the equations)
    temp_ones=np.ones((1,points.shape[1]))
    points=np.concatenate((points, temp_ones),axis=0)  # 4X10

    ##extract object and camera matrices
    K, R_camera_vec, t_camera, R_object_vec, t_object = params

    K=np.concatenate((K,np.zeros((3,1))),axis=1)  #K=3X4

    R_camera_mat=rodrigues(R_camera_vec) #3X3
    R_object_mat=rodrigues(R_object_vec)     #3X3

    temp_arr=np.array([0,0,0,1])[None,:]
    E_camera=np.concatenate((R_camera_mat, t_camera),axis=1)
    E_camera=np.concatenate((E_camera, temp_arr),axis=0)  #4X4

    E_object=np.concatenate((R_object_mat, t_object),axis=1)
    E_object=np.concatenate((E_object, temp_arr),axis=0)   #4X4

    #x=(3X10)
    x = K@E_camera@E_object@points  # (3X4) @ (4X4) @ (4X4) @ (4X10) = (3X10)

    x=x[:2,:]/x[2,:]   # (2X10)

    return(x.T)   # (10X2)

In [ ]:
#params = (K_camera(3X3), R_camera(3X1), t_camera(3X1), R_object(3X1), t_object(3X1))
# fx,fy,Ox,Oy,===Known
# params_video (1d vector) size = (num_framesX12 + 3Xnum_keypoints): R_camera_x,R_camera_y,R_camera_z,
# t_camera_x,t_camera_y,t_camera_z,
# R_object_x,R_object_y,R_object_z,
# t_object_x,t_object_y,t_object_z
# X_w, Y_w, Z_w
#params_video - 1d vector whose elements will be optimised

In [ ]:
## points_3d: global 3d coordinates of each keypoint which will remain same for each frame
# (num_keypoints X 3)
## points_2d: 2d image pixel coordinates of each keypoints across each frame
# (num_frames X num_keypoints X 2)
def reprojection_error(K, params_video, points_2d,num_frames,verbose=False):
    error=[]
    error_mean=[]
    error_linalg=[]
    error_ind=np.array([])
    points_3d=params_video[num_frames*12:].reshape(10,3) #reshape 1d points_3d vector or params_video[-30:]
    for i in range(num_frames):
        ###for a single frame
        params_temp=params_video[i*12:(i*12)+12]
        R_camera=params_temp[:3][:,None]
        t_camera=params_temp[3:6][:,None]
        R_object=params_temp[6:9][:,None]
        t_object=params_temp[9:12][:,None]
        params=(K,R_camera,t_camera,R_object,t_object)
        projections=project(points_3d, params)
        error.append(np.sum((points_2d[i]-projections)**2))
        error_ind=np.append(error_ind,((points_2d[i]-projections)**2).ravel())
        if(verbose==True):
            error_mean.append(np.mean((points_2d[i]-projections)**2))
            error_linalg.append(np.linalg.norm(points_2d[i]-projections))
            print("i: ",i,((points_2d[i]-projections)**2).ravel())
            print("sum of squared residuals for a frame:",error[i])
            print("RMSE for a frame:",np.sqrt(error_mean[i]))
            print("linalg for a frame:",error_linalg[i])
    if(verbose==True):
        plt.plot(error)
        plt.show()
        plt.plot(np.sqrt(error_mean))
        plt.show()
        plt.plot(error_linalg)
        plt.show()
        plt.plot(projections[:,0],projections[:,1],label='reprojected_2d_from_3d_coordinates')
        plt.plot(points_2d[i][:,0],points_2d[i][:,1],label='actual_2d_points')
        plt.legend()
        plt.show()
        print("sum of squared residuals for entire video: ",np.sum(error))
        print("linalg_sum for entire video: ",np.sum(error_linalg))
        print("avg RMSE per frame for entire video: ",np.mean(np.sqrt(error_mean)))
        print("std(RMSE) for video: ",np.std((np.sqrt(error_mean))))
        print("RMSE for entire video: ", np.sqrt(np.mean(error_ind)))
    return(np.mean(error))

In [ ]:
##points_3d (num_keypoints,3)
##points_2d (num_frames, num_keypoints,2)
def bundle_adjustment(K,points_3d, points_2d,num_frames,max_iters):
    initial_params=np.concatenate((np.ones(num_frames*12), points_3d.reshape(-1)))
    error_func = lambda params: reprojection_error(K,params,points_2d,num_frames)
    result = minimize(error_func, initial_params, method='L-BFGS-B', jac='2-point', options={'disp': True,
                                                                                             'maxiter':max_iters,
                                                                                            'maxfun':900000})
    return result.x

In [ ]:
def custom_plot1(x,y,marker,colors,labels):
    plt.scatter(x,y,marker=marker,c=colors,cmap='Accent',label=labels)
    plt.legend()
    for i, (x_value, y_value) in enumerate(zip(x, y), start=0):
        plt.annotate(i, (x_value, y_value), textcoords="offset points", xytext=(0, 10), ha='center')

 

def custom_plot2(x1,y1,x2,y2):
    colors = np.random.rand(len(x1))
    custom_plot1(x1,y1,"*",colors,"actual 2d keypoints")
    custom_plot1(x2,y2,"o",colors,"reprojected 2d keypoints")

def plot_3d_output(points_3d_1,points_3d_2):
    points_3d_1x=points_3d_1[:,0]
    points_3d_1y=points_3d_1[:,1]
    points_3d_1z=points_3d_1[:,2]

    points_3d_2x=points_3d_2[:,0]
    points_3d_2y=points_3d_2[:,1]
    points_3d_2z=points_3d_2[:,2]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points_3d_1x, points_3d_1y, points_3d_1z,color='b',marker='o')
    ax.scatter(points_3d_2x, points_3d_2y, points_3d_2z,color='g',marker='*')

 

    for i, (x_value, y_value, z_value) in enumerate(zip(points_3d_1x, points_3d_1y, points_3d_1z), start=0):
        ax.text(x_value, y_value, z_value, str(i), size=10, zorder=1, color='k')

    for i, (x_value, y_value, z_value) in enumerate(zip(points_3d_2x, points_3d_2y, points_3d_2z), start=0):
        ax.text(x_value, y_value, z_value, str(i), size=10, zorder=1, color='k')

 

    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')
    # plt.show()

 

def interactive_3d(x,y,z):
    trace = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=6,
            color=np.random.rand(len(x)),# set color to an array/list of desired values
            colorscale='Jet',# choose a colorscale
            opacity=0.8
        ),
        text=np.arange(len(x)),
        hoverinfo='text'
    )

 

    data = [trace]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()

def plot_result(num_frame,points_2d, optimised_params):
    points_3d_temp=optimised_params[-30:].reshape(10,3)
    params_temp=optimised_params[num_frame*12:(num_frame*12)+12]
    R_camera=params_temp[:3][:,None]
    t_camera=params_temp[3:6][:,None]
    R_object=params_temp[6:9][:,None]
    t_object=params_temp[9:12][:,None]
    params_temp=(K,R_camera,t_camera,R_object,t_object)
    projections=project(points_3d_temp, params_temp)
    custom_plot2(points_2d[num_frame][:,0],points_2d[num_frame][:,1],projections[:,0],projections[:,1])